Implementation of Group Fourier matrix : 
    $$ 
    G = (\mathbb{Z}_N)^2 \rtimes \mathbb{Z}_4
    $$ 

in order to compute the $G$-convolution efficiently (and gain translation-rotation equivariance)

For now, we implement the case where $N= 2$  (2D image of size 2x2)

In [3]:
import numpy as np
### 
def direct_sum(A, B):
    m, n = A.shape
    p, q = B.shape
    result = np.zeros((m + p, n + q), dtype=np.result_type(A, B))
    result[:m, :n] = A
    result[m:, n:] = B
    return result


def direct_sum_all(*matrices):
    result = np.zeros((0, 0))
    for M in matrices:
        result = direct_sum(result, M)
    return result

def dft_matrix(N):
    """1D DFT matrix of size N (normalized)."""
    omega = np.exp(-2j * np.pi / N)
    return np.array([[omega**(i * j) for j in range(N)] for i in range(N)]) / np.sqrt(N)


In [4]:
## Prepare
DFT_ZN2 = np.kron(dft_matrix(2), dft_matrix(2))

rho_00 = DFT_ZN2[0]
rho_10 = DFT_ZN2[1]
rho_01 = DFT_ZN2[2]
rho_11 = DFT_ZN2[3]


Construction by hands:

In [6]:
omega = -1j


FG_rows = []

for i in range(4):
    FG_rows.append(
        np.hstack([rho_00, (omega**i)*rho_00, (omega**i)**2*rho_00, (omega**i)**3*rho_00])/2,
    )
for i in range(4):
    FG_rows.append(
        np.hstack([rho_11, (omega**i)*rho_11, (omega**i)**2*rho_11, (omega**i)**3*rho_11])/2,
    )

# 1st 2D-irreps
irreps2D_1 = np.zeros((4, 0))
S  = np.array(
    [
        [0, 1],
        [1, 0]
    ]
)
for k in range(4): 
    for i in range(4):
        tmp =  ((np.linalg.matrix_power(S, k))@np.diag([rho_10[i], rho_01[i]])).reshape(-1, 1)
        irreps2D_1 = np.hstack([irreps2D_1, tmp])
irreps2D_1 /= np.sqrt(2)
FG_rows.append(irreps2D_1)

# 2st 2D-irreps
irreps2D_2 = np.zeros((4, 0))
S  = np.array(
    [
        [0, 1],
        [-1, 0]
    ]
)
for k in range(4): 
    for i in range(4):
        tmp =  ((np.linalg.matrix_power(S, k))@np.diag([rho_10[i], rho_01[i]])).reshape(-1, 1)
        irreps2D_2 = np.hstack([irreps2D_2, tmp])
irreps2D_2 /= np.sqrt(2)
FG_rows.append(irreps2D_2)

In [7]:
FG = np.vstack(FG_rows)
FG

array([[ 2.50000000e-01+0.00000000e+00j,  2.50000000e-01+0.00000000e+00j,
         2.50000000e-01+0.00000000e+00j,  2.50000000e-01+0.00000000e+00j,
         2.50000000e-01+0.00000000e+00j,  2.50000000e-01+0.00000000e+00j,
         2.50000000e-01+0.00000000e+00j,  2.50000000e-01+0.00000000e+00j,
         2.50000000e-01+0.00000000e+00j,  2.50000000e-01+0.00000000e+00j,
         2.50000000e-01+0.00000000e+00j,  2.50000000e-01+0.00000000e+00j,
         2.50000000e-01+0.00000000e+00j,  2.50000000e-01+0.00000000e+00j,
         2.50000000e-01+0.00000000e+00j,  2.50000000e-01+0.00000000e+00j],
       [ 2.50000000e-01+0.00000000e+00j,  2.50000000e-01+0.00000000e+00j,
         2.50000000e-01+0.00000000e+00j,  2.50000000e-01+0.00000000e+00j,
         0.00000000e+00-2.50000000e-01j,  0.00000000e+00-2.50000000e-01j,
         0.00000000e+00-2.50000000e-01j,  0.00000000e+00-2.50000000e-01j,
        -2.50000000e-01+0.00000000e+00j, -2.50000000e-01+0.00000000e+00j,
        -2.50000000e-01+0.00000000e+0

In [8]:
# check unitary 
np.linalg.norm(FG @ (FG.conj().T) - np.eye(16))

1.9842463213360646e-15

Permutation : rho_00 and rho_10 are extendable, rho_10, rho_01 forms an orbit under group action

In [10]:
P = np.kron(np.eye(4), 
            np.array(
                [
                    [1, 0, 0, 0],

                ]
            ))

In [11]:
dft_matrix(4)[1]

array([ 5.00000000e-01+0.000000e+00j,  3.06161700e-17-5.000000e-01j,
       -5.00000000e-01-6.123234e-17j, -9.18485099e-17+5.000000e-01j])

In [12]:
DFT_4 = np.kron(dft_matrix(4), np.eye(4))